In [ ]:
import itk
from itkwidgets import view

# Read h5

In [ ]:
import h5py
# read h5py
path = '../datasets/liver_test10.h5'
f = h5py.File(path, 'r')
import pprint, json
print(json.dumps({k: [
    [],
    [],
    [],
    [],
    []
] for k in f.keys()}, indent=4))

In [ ]:
import h5py
# read h5py
path = '../datasets/brain_test_23.h5'
f = h5py.File(path, 'r')
import pprint, json
print(json.dumps({k: [
    [],
    [],
    [],
    [],
    []
] for k in f.keys()}, indent=4))

## Label

In [ ]:
# l4, l3, main, l2, l1
seg_p = {
    "1-0": [
        [],
        [],
        [],
        [],
        []
    ],
    "1-1": [
        [],
        [],
        [],
        [],
        []
    ],
    "2-0": [
        [],
        [],
        [],
        [],
        []
    ],
    "2-1": [
        [],
        [],
        [],
        [],
        []
    ],
    "3-0": [
        [59,64,67],
#         [63,70,55],
        [69,65,67],
        [561,69,54],
        [67,81,41],
        [79,90,50]
    ],
    "3-1": [
        [60,58,70],
        [71,63,74],
        [64,62,53],
        [70,67,44],
        [73,86,39]
    ],
    "4-0": [
        [],
        [],
        [],
        [],
        []
    ],
    "4-1": [
        [],
        [],
        [],
        [],
        []
    ],
    "5-0": [
        [],
        [],
        [],
        [],
        []
    ],
    "5-1": [
        [],
        [],
        [],
        [],
        []
    ]
}
seg_p = {
    "1-1": [
        [],
        [],
        [],
        [],
        []
    ],
    "1-2": [
        [],
        [],
        [],
        [],
        []
    ],
    "2-1": [
        [],
        [],
        [],
        [],
        []
    ],
    "2-2": [
        [],
        [],
        [],
        [],
        []
    ],
    "3-1": [
        [62,43,58],
        [63,86,69],
        [62,39,77],
        [79,67,76],
        [47,32,77],
        [41,86,76]
    ],
    "3-2": [
        [64, 44, 57], # this is the highest intersect
        [64, 83, 71], # this is the posterior intersect
        [75,38,77],
        [85,87,77],
        [47,36,77],
        [43,83,77]
    ],
    "4-1": [
        [],
        [],
        [],
        [],
        []
    ],
    "4-2": [
        [],
        [],
        [],
        [],
        []
    ],
    "5-1": [
        [],
        [],
        [],
        [],
        []
    ],
    "5-2": [
        [],
        [],
        [],
        [],
        []
    ]
}

In [ ]:
def save_h5(points):
    import numpy as np
    f.close()
    fw = h5py.File(path, 'r+')
    for id, p in points.items():
        if len(p[0]):
            print('write point for {}'.format(id))
            pp = np.array(p).round().astype('uint8')
            # flip pp in the last dim
            pp = pp[:, ::-1]
            # create points dataset
            if 'point' in fw[id]:
                del fw[id]['point']
            fw[id].create_dataset('point', data=pp)
    fw.close()
save_h5(seg_p)

In [ ]:
def make_seg(points, r=5):
    seg = np.zeros(data.shape)
    for p in points:
        seg[p[2]-r:p[2]+r,p[1]-r:p[1]+r,p[0]-r:p[0]+r]=1
    return seg
import numpy as np
idx = str('3-1')
data = np.array(f[idx]['volume'])
# label = make_seg(seg_p[idx], 2)
# label = np.array(f[idx]['segmentation'])
(np.array(f[idx]['segmentation'])>1.5).sum()

In [ ]:
view(data)

In [ ]:
# v=view(data, label_image=label)
v = view(data)

In [ ]:
import json
from pprint import pprint
# json.dump(seg_p, fp=open("/home/hynx/regis/recursive-cascaded-networks/landmark_json/lits17_landmark.json", 'w'))

## Nibabel view

In [ ]:
import numpy as np
def visualize_nii(id, path='/mnt/sdc/lits/train'):
    # img, meta = LoadImage()(f'{path}/volume-{id}.nii')
    import nibabel as nib
    from nibabel.orientations import io_orientation, axcodes2ornt, ornt_transform
    imgn : nib.Nifti1Image = nib.load(f'{path}/volume-{id}.nii')
    print(np.array(imgn.dataobj).shape)
    return imgn.orthoview()

In [ ]:
v.position

In [ ]:
id = 128
v=visualize_nii(id)
v.set_position(70, 75, 350)

In [ ]:
id = 39
v=visualize_nii(id)
v.set_position(200, 200, 200)

# Else

In [ ]:
import numpy as np
import pickle as pkl
import ipyvolume.pylab as ipv

In [ ]:
from adet.utils.visualize_niigz import *


def show_img(res):
    import torchvision.transforms as T
    res = tt(res)
    if res.ndim>=3:
        return T.ToPILImage()(visulize_3d(res))
    # normalize res
    res = (res-res.min())/(res.max()-res.min())
    return T.ToPILImage()(res)

## Read Pred pkl and GT files

In [ ]:
fname = '/home/hynx/regis/Recursive-Cascaded-Networks/evaluate/Aug23-1306-model-99500.pkl'
mode = 'normal'
fname = '/home/hynx/regis/Recursive-Cascaded-Networks/evaluate/Aug30-1417-model-99500.pkl'
mode = 'mask'

dct = pkl.load(open(fname, 'rb'))
idx=0
k = 'warped_seg_moving'
p = np.maximum((dct[k][...,0,1]>0.5*255)*2,(dct[k][...,0,0]>0.5*255))
# include img1,2 seg1,2 warped_moving
keys = ['img1', 'img2', 'seg1', 'seg2', 'warped_moving']
seg_pairs = list(zip(*[dct[k][...,0] for k in keys], p))
keys.append(k)
# keys: img1, img2, seg1, seg2, warped_moving, warped_seg_moving
# values: shape(128, 128, 128)
ndct = [{k:v for k,v in zip(keys, s)} for s in seg_pairs]

In [ ]:
import h5py
h5_name = '/home/hynx/regis/Recursive-Cascaded-Networks/datasets/lits_deform_L.h5'
reader = h5py.File(h5_name, 'r')
id1 = dct['id1'][idx][7:]
id2 = dct['id2'][idx]
gt = reader[id1]['id2'][id2]['ffd_gt'][...]
gt_flow = gt - np.mgrid[0:gt.shape[0], 0:gt.shape[1], 0:gt.shape[2]].astype(np.float32).transpose(1,2,3,0)

In [ ]:
img1 = ndct[idx]['img1']
img2 = ndct[idx]['img2']
seg1 = ndct[idx]['seg1']
seg2 = ndct[idx]['seg2']
real_flow = dct['real_flow'][idx]
w_seg2 = ndct[idx]['warped_seg_moving']
from scipy.ndimage import map_coordinates
gt_seg2 = map_coordinates(seg2, gt.transpose(-1,0,1,2), order=0)

In [ ]:
# show_img(map_coordinates(seg2, real_flow.transpose(-1,0,1,2)+np.mgrid[:128,:128,:128], order=0))

In [ ]:
i = 8
x,y,z = np.mgrid[:i, :i, :i]*(128//i)
v = real_flow[x, y, z]
gt_v = gt_flow[x,y,z]
label = w_seg2[x, y, z]
gt_label = gt_seg2[x,y,z]

In [ ]:
flt = lambda x:x.flatten().astype(float)
dist = lambda x:np.linalg.norm(x, axis=-1)

## ITK Tools

In [ ]:
from itkwidgets import view
quant_values = lambda x, values: (x[...,None]>=values).sum(-1)

### Warped grid area

In [ ]:
pred_flow = np.moveaxis(real_flow, -1,0)

In [ ]:
def single_jacobian_det(flow):
    """
    flow has shape (batch, C, H, W, S)
    """
    flow = flow[None]
    # Compute Jacobian determinant
    batch_size, _, height, width, depth = flow.shape
    dx = flow[:, :, 1:, 1:, 1:] - flow[:, :, :-1, 1:, 1:] + 1
    dy = flow[:, :, 1:, 1:, 1:] - flow[:, :, 1:, :-1, 1:] + 1
    dz = flow[:, :, 1:, 1:, 1:] - flow[:, :, 1:, 1:, :-1] + 1
    jac = np.stack([dx, dy, dz], axis=1).transpose(0, 3, 4, 5, 1, 2)
    det = np.linalg.det(jac)
    return det[0]

In [ ]:
areas = abs(single_jacobian_det(pred_flow))
# _, values = np.histogram(areas, bins=5)
values = np.percentile(areas, q=[75, 85, 95])

In [ ]:
 np.histogram(areas, bins=5)

In [ ]:
areas_label = quant_values(areas,values)

In [ ]:
# interpolate to same size
from scipy.ndimage import zoom
areas_label = zoom(areas_label, 128/areas_label.shape[0], order=0)

In [ ]:
np.unique(areas_label, return_counts=True)

In [ ]:
view(img2, areas_label)

In [ ]:
ipv.figure()
N = 1000
x, y, z = np.random.normal(0, 1, (3, N))
ipv.scatter(x,y,z)
# ipv.plot_wireframe(*fixed_loc)
ipv.show()

### Displacement

In [ ]:
disp = dist(real_flow)
# _, values = np.histogram(disp, bins=5)
values = np.percentile(disp, q=[75, 85, 95])

In [ ]:
values

In [ ]:
np.histogram(disp)

In [ ]:
dist_label = quant_values(disp, values)
view(img2, dist_label)

## Animation

In [ ]:
red_color = np.array([255, 0, 0, 200])/255
blue_color = np.array([0, 0, 255, 200])/255

def label2color(label):
    color_label = np.where(label[...,None]==1, red_color, np.array([0.1,0.1,0.1,0.1]))
    color_label = np.where(label[...,None]==2, blue_color, color_label)
    color_label = color_label.astype(float)
    color_label = color_label.reshape(-1,color_label.shape[-1])
    return color_label

x1 = x.flatten().astype(float)
y1 = y.flatten().astype(float)
z1 = z.flatten().astype(float)

In [ ]:
rsh = lambda x:x.reshape(16,16,16)
ipv.figure()
for u,v,w in zip(rsh(x2),rsh(y2),rsh(z2)):
    ipv.plot_wireframe(u, v, w, color="pink")
ipv.show()

In [ ]:
cmap = plt.cm.get_cmap('jet')
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))
plt.figure(figsize=(10,1))
plt.imshow(gradient, cmap=cmap, aspect='auto',)

In [ ]:
vx, vy, vz = v.transpose(-1,0,1,2)
msk = np.logical_or(length_v>5,label==2)
flabel = label.flatten()
x2 = x+vx
y2 = y+vy
z2 = z+vz
x2, y2, z2 = flt(x2), flt(y2), flt(z2)

xs,ys,zs = np.stack([x2,x1]),np.stack([y2,y1]),np.stack([z2,z1])
xs,ys,zs = xs[:,flabel>0], ys[:, flabel>0], zs[:, flabel>0]
fig = ipv.figure()
# get color by displacement and labels
disp = dist(v).flatten()[...,None][flabel>0]
colors = cmap()

s = ipv.scatter(xs,ys,zs, marker='sphere', color=colors, size=2)
ipv.animation_control(s)
ipv.volshow(seg2.transpose(2,1,0)/2,level=[0.5, 1], opacity=[0.01, 0.02], level_width=0.01)
ipv.show()

In [ ]:
# vx, vy, vz = v.transpose(-1,0,1,2)
vx, vy, vz = gt_v.transpose(-1,0,1,2)
x2 = x+vx
y2 = y+vy
z2 = z+vz
x2, y2, z2 = flt(x2), flt(y2), flt(z2)

xs,ys,zs = np.stack([x2,x1]),np.stack([y2,y1]),np.stack([z2,z1])
fig = ipv.figure()
s = ipv.scatter(xs,ys,zs, marker='sphere', color=label2color(gt_label), size=2)
ipv.animation_control(s)
ipv.volshow(seg2.transpose(2,1,0)/2,level=[0.5, 1], opacity=[0.01, 0.02], level_width=0.01)
ipv.show()

In [ ]:
%matplotlib widget
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter(x, y, z, c=color_label.reshape(-1,3), s=3)

## Arrow: Visualize Pred flow

In [ ]:
x = x[msk].flatten().astype(float)
y = y[msk].flatten().astype(float)
z = z[msk].flatten().astype(float)

vx, vy, vz = v[msk].T
vx = vx.flatten()
vy = vy.flatten()
vz = vz.flatten()

In [ ]:
%matplotlib widget
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.gca(projection='3d')

ax.quiver(x, y, z, vx, vy, vz, arrow_length_ratio=0.6, length=1.2, pivot='middle')
ax.scatter(x,y,z, color = 'black', s=0.5)
# show the axis title
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

plt.show()

In [ ]:
fig = ipv.figure()
p1 = np.zeros(3)
p2 = np.ones(3)
p3 = np.full(3, 0.5)
quiver = ipv.quiver(*np.stack([p1,p2, p3],axis=1),[1,10, 500],[1,10,500],[1,10,500], size=5)
ipv.show()

In [ ]:
fig = ipv.figure()
quiver = ipv.quiver(x, y, z, vx, vy, vz, size=5)
ipv.show()